In [7]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("SpendAnalysisLocal") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

In [9]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Converting Cards data to Parquet Format").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [10]:
from pyspark.sql.types import *

#used "_" instead of space to match parqate file format
raw_schema = StructType([
    StructField("User", IntegerType(), True),
    StructField("CARD_INDEX", IntegerType(), True),
    StructField("Card_Brand", StringType(), True),
    StructField("Card_Type", StringType(), True),
    StructField("Card_Number", StringType(), True),
    StructField("Expires", StringType(), True),           
    StructField("CVV", StringType(), True),
    StructField("Has_Chip", StringType(), True),
    StructField("Cards_Issued", IntegerType(), True),
    StructField("Credit_Limit", StringType(), True),      
    StructField("Acct_Open_Date", StringType(), True),
    StructField("Year_PIN_Last_Changed", IntegerType(), True),
    StructField("Card_on_Dark_Web", StringType(), True)
])


In [11]:
path = "file:///home/talentum/shared/dataSource/sd254_cards.csv"
df_raw = spark.read.csv(path, header = True, schema=raw_schema)
df_raw.show()

+----+----------+----------+---------------+----------------+-------+---+--------+------------+------------+--------------+---------------------+----------------+
|User|CARD_INDEX|Card_Brand|      Card_Type|     Card_Number|Expires|CVV|Has_Chip|Cards_Issued|Credit_Limit|Acct_Open_Date|Year_PIN_Last_Changed|Card_on_Dark_Web|
+----+----------+----------+---------------+----------------+-------+---+--------+------------+------------+--------------+---------------------+----------------+
|   0|         0|      Visa|          Debit|4344676511950444|12/2022|623|     YES|           2|      $24295|       09/2002|                 2008|              No|
|   0|         1|      Visa|          Debit|4956965974959986|12/2020|393|     YES|           2|      $21968|       04/2014|                 2014|              No|
|   0|         2|      Visa|          Debit|4582313478255491|02/2024|719|     YES|           2|      $46414|       07/2003|                 2004|              No|
|   0|         3|     

In [12]:
df_raw.write.mode("overwrite").orc('/user/talentum/projectMaster/dataStaging/sd254_cards.parquet')